<a href="https://colab.research.google.com/github/revirevy/RAG_NIX/blob/main/RAG_FOR_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

chatbot


In [1]:
%pip install -q uv PyPDF2 sentence-transformers langchain faiss-cpu groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!uv pip install -U langchain-community -q --system transformers sentence_transformers numpy PyPDF2  langchain faiss-cpu groq --no-cache-dir


In [3]:
!uv pip install -U -q --system sentence_transformers numpy transformers

In [ ]:
!uv pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q --system

In [ ]:
!pip show numpy

In [16]:
%pip show sentence_transformers

Name: sentence-transformers
Version: 3.4.1
Summary: State-of-the-Art Text Embeddings
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by: 


In [1]:
!uv pip show sentence_transformers

Using Python 3.11.11 environment at: /usr
Name: sentence-transformers
Version: 3.4.1
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by:


In [1]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from groq import Groq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
def extract_text_from_pdfs(pdf_folder):
    """
    Extracts text from all PDF files in the given folder.
    """
    text_data = ""
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith('.pdf'):
            reader = PdfReader(os.path.join(pdf_folder, pdf_file))
            for page in reader.pages:
                text_data += page.extract_text()
    return text_data


In [ ]:
def split_text_into_chunks(text, chunk_size=500, chunk_overlap=50):
    """
    Splits text into smaller chunks for embedding generation.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)


In [ ]:
def create_vector_database(chunks, embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Creates a vector database from the given chunks using HuggingFace embeddings.
    """
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store


In [ ]:
def initialize_groq():
    """
    Initializes the Groq client for the chatbot.
    """
    client = Groq(api_key=os.getenv("GROQ_API_KEY"))  # Replace with your API key
    return client


In [ ]:
def chatbot(query, vector_store, groq_client, model="llama-3.3-70b-versatile"):
    """
    Retrieves relevant chunks from the vector database and generates a response using Groq.
    """
    # Retrieve relevant chunks
    docs = vector_store.similarity_search(query, k=5)
    context = " ".join([doc.page_content for doc in docs])

    # Generate response
    chat_completion = groq_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": f"Use the following context for your answer: {context}"},
            {"role": "user", "content": query}
        ]
    )
    return chat_completion.choices[0].message.content


In [ ]:
def initialize_groq():
    """
    Initializes the Groq client for the chatbot.
    """
    # Replace "your_actual_api_key" with your API key
    client = Groq(api_key="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")
    return client

# Example usage of the chatbot
if __name__ == "__main__":
    # Path to folder containing PDFs
    pdf_folder = "/content/pdf"  # Update with your folder path containing PDFs

    # Step 1: Extract text from PDFs
    text_data = extract_text_from_pdfs(pdf_folder)

    # Step 2: Split text into chunks
    chunks = split_text_into_chunks(text_data)

    # Step 3: Create a vector database
    vector_store = create_vector_database(chunks)

    # Step 4: Initialize Groq
    groq_client = initialize_groq()  # No arguments passed here

    # Step 5: Query the chatbot
    query = "What is funtion of ethylene?"
    response = chatbot(query, vector_store, groq_client)

    print("Chatbot Response:", response)


Chatbot Response: According to the text, the functions of ethylene are:

1. Breaking seed and bud dormancy, and initiating germination in peanut seeds and sprouting of potato tubers.
2. Promoting rapid internode/petiole elongation in deep water rice plants, helping leaves/upper parts of the shoot to remain above water.
3. Promoting root growth and root hair formation, increasing the absorption surface of plants.
4. Initiating flowering and synchronizing fruit-set apices, developing shoot buds, and young fruits.
5. Producing new leaves, chloroplasts in leaves, lateral shoot growth, and adventitious shoot formation.
6. Influencing horizontal growth of seedlings, swelling of the axis, and apical hook formation in dicot seedlings.
7. Promoting senescence and abscission of plant organs, especially leaves and flowers.
8. Enhancing the respiration rate during fruit ripening, also known as the "respiratory climactic".
9. Inducing flowering in mango.
10. Hastening fruit ripening in tomatoes and